In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import shutil
import time
import copy
from PIL import Image
import glob
import cv2

In [2]:
filepath = 'mask1_model_resnet101.pth'
model = torch.load(filepath,map_location='cpu')


class_names = ['with_mask',
 'without_mask'
]

In [3]:
def process_image(image):
    
    pil_image = image
   
    image_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    img = image_transforms(pil_image)
    return img

In [4]:

def classify_face(image):
    device = torch.device("cpu")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    im = Image.fromarray(image)
    image = process_image(im)
    print('image_processed')
    img = image.unsqueeze_(0)
    img = image.float()

    model.eval()
    model.cpu()
    output = model(image)
    print(output,'##############output###########')
    _, predicted = torch.max(output, 1)
    print(predicted.data[0],"predicted")


    classification1 = predicted.data[0]
    index = int(classification1)
    print(class_names[index])
    return class_names[index]


In [ ]:
# Check if your camera is working

detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
faces = detector.detectMultiScale(img,1.05)
camera = cv2.VideoCapture(0)
b,img = camera.read()
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
camera.release()
plt.imshow(img)


In [9]:
# Detect Output when space is pressed , exit when esc is pressed

cam = cv2.VideoCapture(0)

cv2.namedWindow("test")

img_counter = 0

while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
    cv2.imshow("test", frame)

    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed
        img_name = "opencv_frame_{}.png".format(img_counter)
        cv2.imwrite(img_name, frame)
        print("{} written!".format(img_name))
        img_counter += 1
        image = cv2.imread(img_name)
        label = classify_face(image)
        print("the label is", label)


cam.release()

cv2.destroyAllWindows()

opencv_frame_0.png written!
image_processed
tensor([[-2.7373,  2.5699]], grad_fn=<AddmmBackward>) ##############output###########
tensor(1) predicted
without_mask
the label is without_mask
opencv_frame_1.png written!
image_processed
tensor([[-2.7627,  2.6004]], grad_fn=<AddmmBackward>) ##############output###########
tensor(1) predicted
without_mask
the label is without_mask
opencv_frame_2.png written!
image_processed
tensor([[-1.2277,  1.1807]], grad_fn=<AddmmBackward>) ##############output###########
tensor(1) predicted
without_mask
the label is without_mask
opencv_frame_3.png written!
image_processed
tensor([[ 0.3517, -0.2959]], grad_fn=<AddmmBackward>) ##############output###########
tensor(0) predicted
with_mask
the label is with_mask
Escape hit, closing...


In [10]:
# Predict in Real Time

camera = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_COMPLEX_SMALL

while True:
    b,img = camera.read()
    if b == False:
        print("Some error")
        continue
    
    
    detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    faces = detector.detectMultiScale(img,1.05)
    for f in faces:
        x,y,w,h = f
        green = (0,255,0)
        red = (0,0,255)
        
        label=classify_face(img)
        if label == 'without_mask':
            cv2.rectangle(img,(x,y),(x+w,y+h),red,2)
        elif label == 'with_mask':
            cv2.rectangle(img,(x,y),(x+w,y+h),green,2)
        cv2.putText(img,str(label),(x,y), font, 1,(255,255,255),1,cv2.LINE_AA)
    
        cv2.imshow("IMAGE",img)

    k = cv2.waitKey(1) & 0xFF
    if k == ord('q'): 
        break
    
    
cv2.destroyAllWindows() 
camera.release()

image_processed
tensor([[-1.0348,  1.0388]], grad_fn=<AddmmBackward>) ##############output###########
tensor(1) predicted
without_mask
image_processed
tensor([[-1.2931,  1.2343]], grad_fn=<AddmmBackward>) ##############output###########
tensor(1) predicted
without_mask
image_processed
tensor([[-0.6082,  0.7031]], grad_fn=<AddmmBackward>) ##############output###########
tensor(1) predicted
without_mask
image_processed
tensor([[-0.2967,  0.2976]], grad_fn=<AddmmBackward>) ##############output###########
tensor(1) predicted
without_mask
image_processed
tensor([[-1.5361,  1.5346]], grad_fn=<AddmmBackward>) ##############output###########
tensor(1) predicted
without_mask
image_processed
tensor([[-2.2304,  2.2041]], grad_fn=<AddmmBackward>) ##############output###########
tensor(1) predicted
without_mask
image_processed
tensor([[-2.0974,  2.0350]], grad_fn=<AddmmBackward>) ##############output###########
tensor(1) predicted
without_mask
image_processed
tensor([[-1.9970,  1.9016]], grad_fn=<A